<a href="https://colab.research.google.com/github/jensullrich/DataScienceSS20/blob/Project/Project/Project_NLP_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Exploring Data

In [1]:
!pip install tensorflow
!pip install --upgrade tensorflow-hub
!pip install bert-for-tf2
!pip install sentencepiece

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.8.0)
     |████████████████████████████████| 40kB 1.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=8ce216597689a982a9fd66dc9775e279a5a8aeb8467716236effd675f1eb7a1f
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=6149bd2d7e09b8c4457342565d35001a2fc7d2cda163a314550f4be4b77c6ec8
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=9ceeabfc547e0afd0b86d11d8b97cbd462ecf8d3920b714040a7e5c417339fbb
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params param

In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone -b Project https://github.com/jensullrich/DataScienceSS20/
  path='DataScienceSS20/Project'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 727 (delta 7), reused 15 (delta 2), pack-reused 705
Receiving objects: 100% (727/727), 133.39 MiB | 26.06 MiB/s, done.
Resolving deltas: 100% (322/322), done.
Checking out files: 100% (223/223), done.


In [3]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import bert

In [4]:
train=pd.read_csv(path + '/DATA/train.csv', index_col='id')
test=pd.read_csv(path + '/DATA/test.csv', index_col='id') # final test dataset for the kaggle competition
submission = pd.read_csv(path + '/DATA/sample_submission.csv')

In [5]:
train

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


## NaN Values

In [6]:
training_has_keyword = train["keyword"].notna().value_counts(normalize=True)
training_has_location = train["location"].notna().value_counts(normalize=True)
try:
    print(f'{training_has_keyword[True]*100:.1f}', "% of training data have keyword,", f'{(training_has_keyword[False])*100:.1f}', "% of values are missing")
    print(f'{training_has_keyword[True]*100:.1f}', "% of training data have location,", f'{(training_has_keyword[False])*100:.1f}', "% of values are missing")

    test_has_keyword = test["keyword"].notna().value_counts(normalize=True)
    test_has_location = test["location"].notna().value_counts(normalize=True)
    print(f'{test_has_keyword[True]*100:.1f}', "% of test data have keyword,", f'{(test_has_keyword[False])*100:.1f}', "% of values are missing")
    print(f'{test_has_location[True]*100:.1f}', "% of test data have location,", f'{(test_has_location[False])*100:.1f}', "% of values are missing")
except KeyError:
    print("NaN values already replaced")

99.2 % of training data have keyword, 0.8 % of values are missing
99.2 % of training data have location, 0.8 % of values are missing
99.2 % of test data have keyword, 0.8 % of values are missing
66.1 % of test data have location, 33.9 % of values are missing


In [7]:
print("Replacing NaN with missing_keyword and missing_location")
train["keyword"] = train["keyword"].fillna('missing_keyword')
train["location"] = train["location"].fillna('missing_location')
test["keyword"] = test["keyword"].fillna('missing_keyword')
test["location"] = test["location"].fillna('missing_location')

Replacing NaN with missing_keyword and missing_location


## Duplicate Values

In [8]:
duplicates = train.groupby(['text']).count().sort_values(by='target', ascending=False)
duplicates = duplicates[duplicates['target']>1]
duplicates.shape

(69, 3)

## Exploration

In [9]:
print("We have", len(train["keyword"]), "tweets with", train["keyword"].nunique(), "unique keywords.")
print("We have", len(train["location"]), "tweets with", train["location"].nunique(), "unique locations.")

We have 7613 tweets with 222 unique keywords.
We have 7613 tweets with 3342 unique locations.


In [10]:
grouped = train.groupby(train["keyword"])
grouped.size().sort_values()

grouped = train.groupby(train["location"])
grouped.target.sum().sort_values(ascending=False).head(10)

location
missing_location    1075
USA                   67
United States         27
Nigeria               22
India                 20
Mumbai                19
UK                    16
New York              16
London                16
Washington, DC        15
Name: target, dtype: int64

# Training

In [11]:
training_parameters = {
    "learning_rate": 0.0001,
    "epochs": 10,
    "batch_size": 32
}

In [12]:
def bert_encode(tweet_texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in tweet_texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [13]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 23.4 s, sys: 7.35 s, total: 30.7 s
Wall time: 33.2 s


In [15]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
FullTokenizer=bert.bert_tokenization.FullTokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [16]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [17]:
model = build_model(bert_layer, max_len=160)
model.summary()

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [28]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=12
)

Epoch 1/3
508/508 [==============================] - 1265s 2s/step - loss: 0.2889 - accuracy: 0.8854 - val_loss: 0.3908 - val_accuracy: 0.8431
Epoch 2/3
508/508 [==============================] - 1265s 2s/step - loss: 0.1860 - accuracy: 0.9333 - val_loss: 0.4485 - val_accuracy: 0.8418
Epoch 3/3
508/508 [==============================] - 1265s 2s/step - loss: 0.0943 - accuracy: 0.9660 - val_loss: 0.5687 - val_accuracy: 0.8431


In [29]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

In [ ]:
def predict_single_tweet(tweet)
    return 0

In [ ]:
def train_disaster_tweets(train):
    train_input = bert_encode(train.text.values, tokenizer, max_len=160)
    return

In [ ]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)